**** データの入手・整形 ****

文に関する極性分析の正解データを用い，以下の要領で正解データ（sentiment.txt）を作成せよ．

rt-polarity.posの各行の先頭に"+1 "という文字列を追加する（極性ラベル"+1"とスペースに続けて肯定的な文の内容が続く）
rt-polarity.negの各行の先頭に"-1 "という文字列を追加する（極性ラベル"-1"とスペースに続けて否定的な文の内容が続く）
上述1と2の内容を結合（concatenate）し，行をランダムに並び替える

sentiment.txtを作成したら，正例（肯定的な文）の数と負例（否定的な文）の数を確認せよ．

In [26]:
# coding: utf-8
import codecs
import random

fname_pos = 'nlp100data/rt-polaritydata/rt-polarity.pos'
fname_neg = 'nlp100data/rt-polaritydata/rt-polarity.neg'
fname_smt = 'nlp100data/sentiment.txt'
fencoding = 'cp1252'        # Windows-1252らしい

In [27]:
result = []

ポジティブデータの読み込み

In [28]:
with codecs.open(fname_pos, 'r', fencoding) as file_pos:
    result.extend(['+1 {}'.format(line.strip()) for line in file_pos])

In [29]:
print("number of postive sentences is {}".format(len(result)))

number of postive sentences is 5331


In [30]:
result_neg = []
with codecs.open(fname_neg, 'r', fencoding) as file_neg:
    result_neg.extend(['-1 {}'.format(line.strip()) for line in file_neg])


In [31]:
print("number of negative sentences is {}".format(len(result_neg)))

number of negative sentences is 5331


In [32]:
result.extend(result_neg)

In [33]:
print("total number of sentences is {}".format(len(result)))

total number of sentences is 10662


シャッフル

In [34]:
random.shuffle(result)

書き出し

In [35]:
with codecs.open(fname_smt, 'w', fencoding) as file_out:
    print(*result, sep='\n', file=file_out)

####  ストップワード

英語のストップワードのリスト（ストップリスト）を適当に作成せよ．

さらに，引数に与えられた単語（文字列）がストップリストに含まれている場合は真，それ以外は偽を返す関数を実装せよ．

さらに，その関数に対するテストを記述せよ．

ストップワードのリスト  http://xpo6.com/list-of-english-stop-words/ のCSV Formatより

In [20]:
stop_words = (
    'a,able,about,across,after,all,almost,also,am,among,an,and,any,are,'
    'as,at,be,because,been,but,by,can,cannot,could,dear,did,do,does,'
    'either,else,ever,every,for,from,get,got,had,has,have,he,her,hers,'
    'him,his,how,however,i,if,in,into,is,it,its,just,least,let,like,'
    'likely,may,me,might,most,must,my,neither,no,nor,not,of,off,often,'
    'on,only,or,other,our,own,rather,said,say,says,she,should,since,so,'
    'some,than,that,the,their,them,then,there,these,they,this,tis,to,too,'
    'twas,us,wants,was,we,were,what,when,where,which,while,who,whom,why,'
    'will,with,would,yet,you,your').lower().split(',')

In [21]:
def is_stopword(str):
    '''文字がストップワードかどうかを返す
    大小文字は同一視する

    戻り値：
    ストップワードならTrue、違う場合はFalse
    '''
    return str.lower() in stop_words


In [25]:
assert is_stopword("Tokyo")

AssertionError: 

In [24]:
assert is_stopword("a")

#### 素性抽出

極性分析に有用そうな素性を各自で設計し，学習データから素性を抽出せよ．

素性としては，レビューからストップワードを除去し，各単語をステミング処理したものが最低限のベースラインとなるであろう．

**** coreNLPを使ったほうがよさそう。nlp100_50.ipynbの先頭を踏襲すればよい（？）****

https://github.com/Lynten/stanford-corenlp

In [40]:
w.strip()

'.'

In [39]:
with codecs.open(fname_smt, 'r', fencoding) as rf:
    for i , l in enumerate(rf):
        for w in l[3:].split(" "):
            print(w.strip())
            
            
        

it
may
be
a
prize
winner
,
but
teacher
is
a
bomb
.
one
of
those
staggeringly
well-produced
,
joylessly
extravagant
pictures
that
keep
whooshing
you
from
one
visual
marvel
to
the
next
,
hastily
,
emptily
.
cold
and
scattered
,
minority
report
commands
interest
almost
solely
as
an
exercise
in
gorgeous
visuals
.
that's
not
vintage
spielberg
and
that
,
finally
,
is
minimally
satisfying
.
at
the
bottom
rung
of
the
series'
entries
.
it's
as
if
de
palma
spent
an
hour
setting
a
fancy
table
and
then
served
up
kraft
macaroni
and
cheese
.
the
experience
of
watching
blobby
old-school
cgi
animation
in
this
superlarge
format
is
just
surreal
enough
to
be
diverting
.
perhaps
no
picture
ever
made
has
more
literally
showed
that
the
road
to
hell
is
paved
with
good
intentions
.
even
when
foreign
directors
.
.
.
borrow
stuff
from
hollywood
,
they
invariably
shake
up
the
formula
and
make
it
more
interesting
.
the
script's
snazzy
dialogue
establishes
a
realistic
atmosphere
that
involves
us
in
the
unfolding
c

tabloid
journalism
.
its
engaging
simplicity
is
driven
by
appealing
leads
.
a
potent
allegorical
love
story
.
montias
.
.
.
pumps
a
lot
of
energy
into
his
nicely
nuanced
narrative
and
surrounds
himself
with
a
cast
of
quirky
--
but
not
stereotyped
--
street
characters
.
schnitzler
does
a
fine
job
contrasting
the
sleekness
of
the
film's
present
with
the
playful
paranoia
of
the
film's
past
.
this
is
the
sort
of
low-grade
dreck
that
usually
goes
straight
to
video
--with
a
lousy
script
,
inept
direction
,
pathetic
acting
,
poorly
dubbed
dialogue
and
murky
cinematography
,
complete
with
visible
boom
mikes
.
this
is
not
an
easy
film
.
but
it
could
be
,
by
its
art
and
heart
,
a
necessary
one
.
everything
about
girls
can't
swim
,
even
its
passages
of
sensitive
observation
,
feels
secondhand
,
familiar
--
and
not
in
a
good
way
.
thoughtless
,
random
,
superficial
humour
and
a
lot
of
very
bad
scouse
accents
yes
,
mibii
is
rote
work
and
predictable
,
but
with
a
philosophical
visual
coming
right
at

the
events
for
posterity
,
but
to
help
us
clearly
see
the
world
of
our
making
.
what
sets
ms
.
birot's
film
apart
from
others
in
the
genre
is
a
greater
attention
to
the
parents
--
and
particularly
the
fateful
fathers
--
in
the
emotional
evolution
of
the
two
bewitched
adolescents
.
a
dreary
movie
.
some
of
the
visual
flourishes
are
a
little
too
obvious
,
but
restrained
and
subtle
storytelling
,
and
fine
performances
make
this
delicate
coming-of-age
tale
a
treat
.
a
quiet
family
drama
with
a
little
bit
of
romance
and
a
dose
of
darkness
.
in
this
film
,
aussie
david
caesar
channels
the
not-quite-dead
career
of
guy
ritchie
.
the
mothman
prophecies
is
best
when
illustrating
the
demons
bedevilling
the
modern
masculine
journey
.
even
accepting
this
in
the
right
frame
of
mind
can
only
provide
it
with
so
much
leniency
.
ultimately
,
the
message
of
trouble
every
day
seems
to
be
that
all
sexual
desire
disrupts
life's
stasis
.
the
master
of
disguise
falls
under
the
category
of
'should
have
been
a


any
sting
to
it
,
as
if
woody
is
afraid
of
biting
the
hand
that
has
finally
,
to
some
extent
,
warmed
up
to
him
.
the
bland
outweighs
the
nifty
,
and
cletis
tout
never
becomes
the
clever
crime
comedy
it
thinks
it
is
.
ultimately
engages
less
for
its
story
of
actorly
existential
despair
than
for
its
boundary-hopping
formal
innovations
and
glimpse
into
another
kind
of
chinese
'cultural
revolution
.
'
snow
dogs
finds
its
humour
in
a
black
man
getting
humiliated
by
a
pack
of
dogs
who
are
smarter
than
him
although
sensitive
to
a
fault
,
it's
often
overwritten
,
with
a
surfeit
of
weighty
revelations
,
flowery
dialogue
,
and
nostalgia
for
the
past
and
roads
not
taken
.
propelled
not
by
characters
but
by
caricatures
.
this
film
was
made
by
and
for
those
folks
who
collect
the
serial
killer
cards
and
are
fascinated
by
the
mere
suggestion
of
serial
killers
.
for
the
rest
of
us
,
sitting
through
dahmer's
two
hours
amounts
to
little
more
than
punishment
.
i
like
it
.
there
is
a
freedom
to
watching


the
movie
starts
with
a
legend
and
ends
with
a
story
that
is
so
far-fetched
it
would
be
impossible
to
believe
if
it
weren't
true
.
this
is
the
stuff
that
disney
movies
are
made
of
.
like
its
new
england
characters
,
most
of
whom
wander
about
in
thick
clouds
of
denial
,
the
movie
eventually
gets
around
to
its
real
emotional
business
,
striking
deep
chords
of
sadness
.
one
of
the
most
splendid
entertainments
to
emerge
from
the
french
film
industry
in
years
.
the
star
who
helped
give
a
spark
to
"
chasing
amy
"
and
"
changing
lanes
"
falls
flat
as
thinking
man
cia
agent
jack
ryan
in
this
summer's
new
action
film
,
"
the
sum
of
all
fears
.
"
it
arrives
with
an
impeccable
pedigree
,
mongrel
pep
,
and
almost
indecipherable
plot
complications
.
lee's
achievement
extends
to
his
supple
understanding
of
the
role
that
brown
played
in
american
culture
as
an
athlete
,
a
movie
star
,
and
an
image
of
black
indomitability
.
almost
nothing
else
--
raunchy
and
graphic
as
it
may
be
in
presentation
--
is
o

...

the
rapidly
changing
face
of
beijing
.
leigh's
daring
here
is
that
without
once
denying
the
hardscrabble
lives
of
people
on
the
economic
fringes
of
margaret
thatcher's
ruinous
legacy
,
he
insists
on
the
importance
of
those
moments
when
people
can
connect
and
express
their
love
for
each
other
.
if
you're
down
for
a
silly
hack-and-slash
flick
,
you
can
do
no
wrong
with
jason
x
.
great
character
interaction
.
the
kids
in
the
audience
at
the
preview
screening
seemed
bored
,
cheering
the
pratfalls
but
little
else
;
their
parents
,
wise
folks
that
they
are
,
read
books
.
too
bad
writer-director
adam
rifkin
situates
it
all
in
a
plot
as
musty
as
one
of
the
golden
eagle's
carpets
.
supposedly
authentic
account
of
a
historical
event
that's
far
too
tragic
to
merit
such
superficial
treatment
.
very
well
made
,
but
doesn't
generate
a
lot
of
tension
.
much
of
the
movie's
charm
lies
in
the
utter
cuteness
of
stuart
and
margolo
.
their
computer-animated
faces
are
very
expressive
.
an
artsploitation
mov

.
.
.
is
funny
in
the
way
that
makes
you
ache
with
sadness
(
the
way
chekhov
is
funny
)
,
profound
without
ever
being
self-important
,
warm
without
ever
succumbing
to
sentimentality
.
the
pretensions
--
and
disposable
story
--
sink
the
movie
.
and
diesel
isn't
the
actor
to
save
it
.
too
timid
to
bring
a
sense
of
closure
to
an
ugly
chapter
of
the
twentieth
century
.
with
miscast
leads
,
banal
dialogue
and
an
absurdly
overblown
climax
,
killing
me
softly
belongs
firmly
in
the
so-bad-it's-good
camp
.
sunk
by
way
too
much
indulgence
of
scene-chewing
,
teeth-gnashing
actorliness
.
it's
both
degrading
and
strangely
liberating
to
see
people
working
so
hard
at
leading
lives
of
sexy
intrigue
,
only
to
be
revealed
by
the
dispassionate
gantz
brothers
as
ordinary
,
pasty
lumpen
.
routine
and
rather
silly
.
rymer
doesn't
trust
laughs
--
and
doesn't
conjure
proper
respect
for
followers
of
the
whole
dead-undead
genre
,
who
deserve
more
from
a
vampire
pic
than
a
few
shrieky
special
effects
.
isabelle


majority
of
the
film
that
shows
a
stationary
camera
on
a
subject
that
could
be
mistaken
for
giving
a
public
oration
,
rather
than
contributing
to
a
film's
narrative
.
a
pure
participatory
event
that
malnourished
intellectuals
will
gulp
down
in
a
frenzy
.
what
is
100%
missing
here
is
a
script
of
even
the
most
elemental
literacy
,
an
inkling
of
genuine
wit
,
and
anything
resembling
acting
.
brings
awareness
to
an
issue
often
overlooked
--
women's
depression
.
the
new
guy
does
have
a
heart
.
now
,
if
it
only
had
a
brain
.
kinnear
doesn't
aim
for
our
sympathy
,
but
rather
delivers
a
performance
of
striking
skill
and
depth
.
a
hypnotic
cyber
hymn
and
a
cruel
story
of
youth
culture
.
.
.
.
a
solid
,
well-formed
satire
.
the
darker
elements
of
misogyny
and
unprovoked
violence
suffocate
the
illumination
created
by
the
two
daughters
and
the
sparse
instances
of
humor
meant
to
shine
through
the
gloomy
film
noir
veil
.
lacks
the
visual
flair
and
bouncing
bravado
that
characterizes
better
hip-hop
c

a
substitute
for
humor
.
a
sly
game
of
cat
and
mouse
that's
intense
and
thrilling
at
times
,
but
occasionally
stretches
believability
to
its
limits
and
relies
on
predictable
plot
contrivances
.
without
ever
becoming
didactic
,
director
carlos
carrera
expertly
weaves
this
novelistic
story
of
entangled
interrelationships
and
complex
morality
.
the
problem
is
the
needlessly
poor
quality
of
its
archival
prints
and
film
footage
.
the
images
lack
contrast
,
are
murky
and
are
frequently
too
dark
to
be
decipherable
.
labute
was
more
fun
when
his
characters
were
torturing
each
other
psychologically
and
talking
about
their
genitals
in
public
.
a
special
kind
of
movie
,
this
melancholic
film
noir
reminded
me
a
lot
of
memento
.
.
.
what's
surprising
about
this
traditional
thriller
,
moderately
successful
but
not
completely
satisfying
,
is
exactly
how
genteel
and
unsurprising
the
execution
turns
out
to
be
.
asks
what
truth
can
be
discerned
from
non-firsthand
experience
,
and
specifically
questions


enjoyable
.
.
.
a
pathetic
exploitation
film
that
tries
to
seem
sincere
,
and
just
seems
worse
for
the
effort
.
certain
to
be
distasteful
to
children
and
adults
alike
,
eight
crazy
nights
is
a
total
misfire
.
is
office
work
really
as
alienating
as
'bartleby'
so
effectively
makes
it
?
we
could
have
expected
a
little
more
human
being
,
and
a
little
less
product
.
there's
not
a
comedic
moment
in
this
romantic
comedy
.
impeccably
filmed
,
sexually
charged
,
but
ultimately
lacking
in
substance
,
not
to
mention
dragged
down
by
a
leaden
closing
act
.
offensive
in
the
way
it
exploits
the
hot-button
issue
of
domestic
abuse
for
cheap
thrills
and
disgusting
in
the
manner
it
repeatedly
puts
a
small
child
in
jeopardy
,
treating
her
as
little
more
than
a
prop
to
be
cruelly
tormented
.
not
always
too
whimsical
for
its
own
good
(
but
enough
to
do
harm
)
,
this
strange
hybrid
of
crime
thriller
,
quirky
character
study
,
third-rate
romance
and
female
empowerment
fantasy
never
really
finds
the
tonal
or
t

In [41]:
from stanfordcorenlp import StanfordCoreNLP

In [42]:
corenlp = StanfordCoreNLP(r'/home/toshinao/Tools/stanford-corenlp-full-2018-02-27')

In [52]:
from bs4 import BeautifulSoup

In [ ]:
props={'annotators': 'lemma','pipelineLanguage':'en','outputFormat':'xml'}

In [77]:
xmls = list()
with codecs.open(fname_smt, 'r', fencoding) as rf:
    for l in rf:
        run_xml = corenlp.annotate(l , properties=props)
        xmls.append(run_xml)
            
        

In [78]:
len(xmls)

10662

In [92]:
"x" not in stop_words

True

In [98]:
from collections import Counter

In [99]:
word_counter = Counter()

In [100]:
for x in xmls:
    soup = BeautifulSoup(x, 'lxml')
    for x in soup.find_all("lemma"):
        y = x.get_text()
        if y not in stop_words:
            word_counter.update([y])
            

In [101]:
features = [word for word, count in word_counter.items() if count >= 6]

In [102]:
features[0:10]

['critic',
 'goofy',
 'disjointed',
 'originality',
 'service',
 'queen',
 'magic',
 'boldly',
 'convenient',
 'safe']

##### sandbox

In [95]:
features = set()

for x in xmls:
    soup = BeautifulSoup(x, 'lxml')
    for x in soup.find_all("lemma"):
        y = x.get_text()
        if y not in stop_words:
            features.add(y)
            

In [86]:
for x in soup.find_all("lemma"):
    print(x.get_text())

-1
devoid
of
any
of
the
quality
that
make
the
first
film
so
special
.


In [50]:
run_xml = corenlp.annotate(l, properties=props)

In [74]:
print(soup.prettify())

<html>
 <body>
  <p>
   Request is too long to be handled by server: 1259823 characters. Max length is 100000 characters.
  </p>
 </body>
</html>


In [43]:
corenlp.parse(fname_smt)

'(ROOT\n  (NP (NN nlp100data/sentiment) (. .)))'

In [44]:
corenlp.parse(l)

'(ROOT\n  (NP\n    (NP (CD -1))\n    (ADJP (JJ devoid)\n      (PP (IN of)\n        (NP\n          (NP (DT any))\n          (PP (IN of)\n            (NP\n              (NP (DT the) (NNS qualities))\n              (SBAR\n                (WHNP (WDT that))\n                (S\n                  (VP (VBD made)\n                    (NP\n                      (NP (DT the) (JJ first) (NN film))\n                      (ADJP (RB so) (JJ special)))))))))))\n    (. .)))'

In [45]:
corenlp.parse?

https://github.com/Lynten/stanford-corenlp

In [47]:
props={'annotators': 'tokenize,ssplit,pos','pipelineLanguage':'en','outputFormat':'xml'}
print(corenlp.annotate(l, properties=props))

<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet href="CoreNLP-to-HTML.xsl" type="text/xsl"?>
<root><document><sentences><sentence id="1"><tokens><token id="1"><word>-1</word><CharacterOffsetBegin>0</CharacterOffsetBegin><CharacterOffsetEnd>2</CharacterOffsetEnd><POS>CD</POS></token><token id="2"><word>devoid</word><CharacterOffsetBegin>3</CharacterOffsetBegin><CharacterOffsetEnd>9</CharacterOffsetEnd><POS>JJ</POS></token><token id="3"><word>of</word><CharacterOffsetBegin>10</CharacterOffsetBegin><CharacterOffsetEnd>12</CharacterOffsetEnd><POS>IN</POS></token><token id="4"><word>any</word><CharacterOffsetBegin>13</CharacterOffsetBegin><CharacterOffsetEnd>16</CharacterOffsetEnd><POS>DT</POS></token><token id="5"><word>of</word><CharacterOffsetBegin>17</CharacterOffsetBegin><CharacterOffsetEnd>19</CharacterOffsetEnd><POS>IN</POS></token><token id="6"><word>the</word><CharacterOffsetBegin>20</CharacterOffsetBegin><CharacterOffsetEnd>23</CharacterOffsetEnd><POS>DT</POS></token><toke